# 데이터셋 분석 및 모델 생성 코드 분석

### 요약

본 문서는 병 판별 프로젝트를 위해 만든 데이터셋을 분석하는 과정과, 데이터 모델 제작에 관한 코드를 모아 놓은 보고서입니다.

### 목차

1. 데이터셋 분석
    - 커스텀 모델 데이터셋
    - 데이터셋 라벨 분류
2. 데이터모델 분석
    - 커스텀 모델 설계
        - 공통목표
        - 1단계
        - 2단계
        - 3단계
        - 이미지 캡셔닝

## 1. 데이터셋 분석

#### 커스텀 모델 데이터셋

- 링크 : https://aihub.or.kr/aihubdata/data/view.do?currMenu=115&topMenu=100&dataSetSn=71385
- AI-Hub의 생활 폐기물 이미지 데이터 원본 551,562장 중에서, 원본 이미지와 함께 있는 JSON 파일의 태그를 활용하여 병 이미지가 있는 이미지를 1차 판별하고, 그 중에서 JSON 파일만 있거나 사진만 있는 파일들을 솎아내는 2차 판별을 시행하여, 총 72,000장의 병 이미지를 분류했습니다.
- 분류된 이미지들을 데이터셋의 JSON label에 따라 9종으로 구분하여 각 폴더에 담은 뒤, 각 이미지를 JSON의 BBOX 좌표를 활용하여 병이 있는 부분을 잘라낸 뒤, 잘라낸 이미지를 224x224 크기로 변환하여, 예시를 위해 각 100장씩을 sample_data 폴더에 담아 놓았습니다.

#### 데이터셋 라벨 분류

- 데이터셋의 JSON 라벨은 다음과 같이 9종류로 분류되어 있었고, 라벨의 정확한 분류는 다음과 같습니다.

|JSON 라벨|실제 라벨|
|------|---|
|c_4_01_01|재사용 유리 + 다중포장재|
|c_4_01_02|재사용 유리(소주병+맥주병)|
|c_4_02_01_01|갈색 유리 + 다중포장재|
|c_4_02_01_02|갈색 유리|
|c_4_02_02_01|녹색 유리 + 다중포장재|
|c_4_02_02_02|녹색 유리|
|c_4_02_03_01|백색 유리 + 다중포장재|
|c_4_02_03_02|백색 유리|
|c_4_03|기타 유리|

# 2. 데이터모델 분석

#### 커스텀 모델 설계

- 공통적으로 모든 모델은 x값으로 이미지를 받으며, 각 모델별로 반환하는 y값은 각 단계별로 적을 예정. 모든 모델은 pytorch를 활용해 훈련했습니다.

- 1단계. cnn으로 데이터셋 라벨 기준에 따라 병 종류(색깔)를 파악하여, 색깔과 bbox값을 반환하도록 하기. 상세 코드는 아래 칸에 작성했습니다.

In [8]:
# 패키지 다운로드
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
import torchvision.transforms as transforms
import torchvision.datasets as datasets

# CUDA 지원 활성화
if torch.cuda.is_available():
    device = torch.device("cuda")
    print("CUDA 사용 가능")
else:
    device = torch.device("cpu")
    print("CUDA 사용 불가")

# 시드 고정
seed = 42
torch.manual_seed(seed)
# GPU 절약
torch.backends.cudnn.benchmark = True

# print(torch.cuda.device_count()) # 사용 가능한 장치가 몇 개인지 확인합니다.
# print(torch.cuda.get_device_name(0)) # 첫번째 GPU의 장치명을 확인합니다.
# print(torch.cuda.get_device_name(1)) # 두번째 GPU의 장치명을 확인합니다.


CUDA 사용 가능


In [9]:
# 이미지 라벨 생성
def get_label_from_foldername(foldername):
    # 각 폴더의 이름에 따라 라벨을 할당
    if foldername == "preprocessed_brown_glass":
        return "0"
    elif foldername == "preprocessed_brown_glass_packaging":
        return "1"
    elif foldername == "preprocessed_clear_glass":
        return "2"
    elif foldername == "preprocessed_clear_glass_packaging":
        return "3"
    elif foldername == "preprocessed_green_glass":
        return "4"
    elif foldername == "preprocessed_green_glass_packaging":
        return "5"
    elif foldername == "preprocessed_reused_glass":
        return "6"
    elif foldername == "preprocessed_reused_glass_packaging":
        return "7"
    elif foldername == "preprocessed_unclassified_glass":
        return "8"
    else:
        raise ValueError(f"Invalid folder name: {foldername}")

In [10]:
# 데이터셋 경로
dataset_path = "preprocessed_image"

# 데이터셋 불러오기
dataset = datasets.ImageFolder(
    dataset_path,
    # 데이터 증강을 위한 변환 함수들 정의
    transform = transforms.Compose([
        transforms.Resize((224, 224)),  # 이미지 크기 조정
        # transforms.RandomResizedCrop(size=224),  # 무작위로 잘라내고 크기 조정
        transforms.RandomHorizontalFlip(),  # 수평으로 무작위로 뒤집기
        transforms.RandomRotation(degrees=30),  # 무작위로 회전 (±30도 범위)
        transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.2),  # 색감 조정
        transforms.ToTensor(),  # 이미지를 텐서로 변환
        transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])  # 이미지 정규화
    ])
)

# 전체 데이터셋을 훈련 데이터와 검증 데이터로 나누기
train_ratio = 0.8
train_size = int(train_ratio * len(dataset))
val_size = len(dataset) - train_size

train_dataset, val_dataset = torch.utils.data.random_split(dataset, [train_size, val_size])

# 데이터로더 생성
batch_size = 4
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_dataloader = DataLoader(val_dataset, batch_size=batch_size)

In [11]:
# 모델 레이어 생성
class Model(nn.Module):
    def __init__(self):
        super(Model, self).__init__()
        self.conv0 = nn.Conv2d(3, 3, kernel_size=1)  # 추가된 레이어
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3)
        self.pool = nn.MaxPool2d(2, 2)
        self.flatten = nn.Flatten()
        self.fc1 = nn.Linear(32 * 111 * 111, 64)
        self.fc2 = nn.Linear(64, 9)  # 클래스 수에 맞게 계속 수정
        
    def forward(self, x):
        x = self.pool(nn.functional.relu(self.conv1(x)))
        x = self.flatten(x)
        x = nn.functional.relu(self.fc1(x))
        x = torch.sigmoid(self.fc2(x))
        return x.to(device)

# 모델 인스턴스 생성 및 CUDA 장치로 이동
model = Model().to(device)

In [12]:
print(model)
print(device)

Model(
  (conv0): Conv2d(3, 3, kernel_size=(1, 1), stride=(1, 1))
  (conv1): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (fc1): Linear(in_features=394272, out_features=64, bias=True)
  (fc2): Linear(in_features=64, out_features=9, bias=True)
)
cuda


In [13]:
# 모델 학습시키기
# 손실 함수와 최적화 알고리즘 설정
criterion = nn.CrossEntropyLoss().to(device)  # 손실 함수 수정
optimizer = optim.Adam(model.parameters(), lr=0.003)

# 학습 반복
epochs = 10
best_accuracy = 0.0  # 가장 좋은 정확도를 저장할 변수 초기화
best_model_path = "best_model.pt" # 가장 좋은 모델 매개변수의 저장 경로
full_model_path = "full_model.pt" # 모델 전체를 저장할 경로 

for epoch in range(epochs):
    running_loss = 0.0
    for images, labels in train_dataloader:
        # 이미지와 라벨을 CUDA 장치로 이동
        images = images.to(device)
        labels = labels.to(device)
        # 그래디언트 초기화
        optimizer.zero_grad()
        # 모델에 이미지 전달하여 예측 수행
        outputs = model(images)
        # 손실 계산
        loss = criterion(outputs, labels)

        # # 가중치 감소 (L2 규제)
        # for param in model.parameters():
        #     loss += 0.001 * torch.norm(param)

        # 역전파 및 가중치 업데이트
        loss.backward()
        optimizer.step()
        running_loss += loss.item()

    print(f"Epoch {epoch+1} - Loss: {running_loss / len(train_dataloader)}")


Epoch 1 - Loss: 2.2335100510743433
Epoch 2 - Loss: 2.23347408177268
Epoch 3 - Loss: 2.233474081620719
Epoch 4 - Loss: 2.2334740818366634
Epoch 5 - Loss: 2.233474081748686
Epoch 6 - Loss: 2.233474081764682
Epoch 7 - Loss: 2.233474081716694
Epoch 8 - Loss: 2.2334740817006984
Epoch 9 - Loss: 2.233474081852659
Epoch 10 - Loss: 2.2334740817886756


In [14]:
# 검증 데이터셋을 통한 모델 평가
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in val_dataloader:
        # 이미지와 라벨을 CUDA 장치로 이동
        images = images.to(device)
        labels = labels.to(device)

        # 모델에 이미지 전달하여 예측 수행
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)  # 예측 수정

        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    accuracy = 100 * correct / total
    print(f"Test Accuracy: {accuracy}%")
    
    # 가장 좋은 정확도를 가진 모델 저장
    if accuracy > best_accuracy:
        best_accuracy = accuracy
        torch.save(model, full_model_path)
        torch.save(model.state_dict(), best_model_path)
        print("Best model saved.")

Test Accuracy: 13.968466957396847%
Best model saved.


- 2. cnn과 lstm을 활용해 더 정확히 판별할 수 있도록 업데이트

- 3. 깨진 병 이미지, 상표 이미지 데이터를 구해서 병의 종류와 만든 회사, 파손 여부를 반환할 수 있게 하기